## <center> *Модель прогнозирования стоимости жилья для агентства недвижимости*

# <center> **Часть II. Разведывательный анализ и моделирование.**

Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.preprocessing as pp 
import sklearn.model_selection as ms 
import sklearn.feature_selection as fs

import sklearn.linear_model as lm
import sklearn.tree as tree
import sklearn.ensemble as ens
import sklearn.svm as svm
import sklearn.metrics as m

import lightgbm as lgbm
import xgboost as xgb
import catboost as cb 

from functions import *
import pickle 

import warnings
warnings.filterwarnings('ignore')

plt.style.use('bmh')

R = 7
np.random.seed(R)

## **Данные**

Читаем данные

In [ ]:
data = pd.read_csv('data/data_edited.csv')

data

In [ ]:
get_data_info(data)

## **Разведывательный анализ**

### Исследование бинарных переменных

In [ ]:
bin_cols = set()
num_cols = []

for col in data.columns:
    if ('target' not in col) and (data[col].nunique() == 2):
        bin_cols.add(col.split()[0])
    else:
        num_cols.append(col)

bin_cols, num_cols

In [ ]:
def show_barplot(cols, data=data, figsize=(20, 5)):
    sums = get_sums(data, cols).sort_values(ascending=False)

    col_list = sums.index.tolist() + ['all']
    vls_list = sums.values.tolist() + [data.shape[0]]

    for i in range(len(col_list)):
        col_list[i] = col_list[i].replace(cols+' ', '')

    df = (pd.DataFrame({'labels': col_list, 
                        'count': vls_list})
          .sort_values('count', ascending=False))
    
    fig, ax = plt.subplots(figsize=figsize)
    sns.barplot(df, x='labels', y='count', hue='labels', ax=ax)
    ax.tick_params(rotation=30)
    ax.set_title(cols)
    
    return fig 


for col in bin_cols:
    show_barplot(col).show()

### Проверка на нормальность

In [ ]:
def test_normality(labels, 
                   data=data, 
                   testfunc='shapiro', 
                   alpha=0.05):
    
    norm_df = pd.DataFrame(index=labels)
    
    pv_list = []
    for col in data[labels].columns:
        x = data[col]
        
        if testfunc == 'dagostino':
            p = sps.normaltest(x)
            pv = round(p.pvalue[0], 3)
            alpha = alpha / 2

        p = sps.shapiro(x)
        pv = round(p.pvalue, 3)
        
        pv_list.append(pv)
    
    norm_df['p_value'] = pv_list
    norm_df['is_normal'] = norm_df['p_value'].apply(lambda pv: 'normal' if pv > alpha else 'not normal')

    return norm_df

In [ ]:
test_normality(num_cols)

### Выявление и очистка выбросов 

In [ ]:
def clean_data(features, data=data, q_limit=0.99, show_plot=False):
    cleaned = data.copy()
    
    for feature in features:
        x = cleaned[feature]

        lim = x.quantile(q_limit)
        cleaned = cleaned[x <= lim][x >= -lim].reset_index(drop=True)
        
    print(f'Shape of cleaned data is {cleaned.shape}.')
    print(f'{data.shape[0] - cleaned.shape[0]} outliers were dropped.')
        
    if show_plot:
        fig, ax = plt.subplots(2, 1, figsize=(15, 15))
        
        sns.kdeplot(data[features], ax=ax[0])
        ax[0].set_ylabel('Original distribution')

        sns.kdeplot(cleaned[features], ax=ax[1])
        ax[1].set_ylabel('Cleaned distribution')

        fig.show()
    
    return cleaned

cleaned_data = clean_data(num_cols, show_plot=True)

### Проверка на мультиколлинеарность

In [ ]:
def get_correlations(method, data=cleaned_data):
    corr_data = (data.corr(method=method)
                 [data.corr(method=method).abs() > 0.9]
                 .round(2))

    for i in corr_data.index:
        for c in corr_data.columns:
            if i == c:
                corr_data.loc[i, c] = np.nan

    for i, c in zip(corr_data.index, 
                    corr_data.columns):
        if corr_data.loc[i].sum() == 0:
            corr_data.drop(i, axis=0, inplace=True)
        if corr_data.loc[:, c].sum() == 0:
            corr_data.drop(c, axis=1, inplace=True)
            
    return corr_data


fig, ax = plt.subplots(1, 2, figsize=(30, 10))

sns.heatmap(get_correlations('pearson'), 
            annot=True, 
            cmap='coolwarm', 
            linewidths=.5, 
            linecolor='grey', 
            ax=ax[0])
ax[0].set_title("Pearson's correlation")

sns.heatmap(get_correlations('spearman'), 
            annot=True, 
            cmap='coolwarm', 
            linewidths=.5, 
            linecolor='grey', 
            ax=ax[1])
ax[1].set_title("Spearman's correlation")
    
fig.show()

In [ ]:
multicollinear_cols = ['heating forced', 'cooling system', 'parking none', 
                       'parking door opener', 'city importance']

cleaned_data.drop(multicollinear_cols, axis=1, inplace=True)

cleaned_data.shape 

### Исследования по выборкам

In [ ]:
def binary_stattest(data, alpha=0.05):
    bins_list = []

    for col in data.columns:
        if data[col].nunique() == 2:
            bins_list.append(col)
            
    statdata = pd.DataFrame(index=bins_list)
    pv_list = []
    dep_list = []
            
    for col_ in bins_list:
        sample_1 = data[data[col_] == 1]
        sample_0 = data[data[col_] == 0]
        
        _, p = sps.mannwhitneyu(sample_0['target'], 
                                sample_1['target'])
        pv_list.append(p)
        
        if p > alpha:
            dep_list.append(0)
        else:
            dep_list.append(1)
            
    statdata['pvalue'] = pv_list
    statdata['depends'] = dep_list
    
    return statdata


bs_data = binary_stattest(cleaned_data)

useless_df = bs_data[bs_data['depends'] == 0]

useless_df

In [ ]:
cleaned_data.drop(useless_df.index, axis=1, inplace=True)

cleaned_data.shape

### Исследование числовых переменных

In [ ]:
num_cols = []

for col in cleaned_data.columns:
    if cleaned_data[col].nunique() > 2:
        num_cols.append(col)

num_cols.remove('target')

numdata_info = get_data_info(cleaned_data[num_cols])

numdata_info

#### *Категориальные*

In [ ]:
cat_cols = numdata_info[numdata_info['Uniques'] <= 20].index.tolist()

cat_cols

In [ ]:
def multiple_stattest(data, cols=cat_cols, alpha=0.05):
    statdata = pd.DataFrame(index=cols)
    pv_list = []
    dep_list = []
    
    for c in cols:
        samples = []

        for i in data[c].unique():
            sample = data[data[c] == i]
            samples.append(sample['target'])
            
        _, p = sps.kruskal(*samples)
        
        pv_list.append(p)
        
        
        if p > alpha:
            dep_list.append(0)
        else:
            dep_list.append(1)
            
    statdata['pvalue'] = pv_list
    statdata['depends'] = dep_list
    
            
    return statdata


multiple_stattest(cleaned_data)

#### *Непрерывные*

In [ ]:
for cat in cat_cols:
    num_cols.remove(cat)
    
num_cols, len(num_cols) 

In [ ]:
fig, ax = plt.subplots(6, 2, figsize=(20, 30))

for i, col in enumerate(num_cols):
    sample = cleaned_data[[col, 'target']].sample(1000, random_state=R)

    if i % 2 == 0:
        ax_place = ax[i//2, 0]
    else:
        ax_place = ax[i//2, 1]
        
    sns.scatterplot(sample, x=col, y='target', ax=ax_place)
    
fig.show()

### Масштабирование данных

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(20, 20))

num_cols += cat_cols

sns.kdeplot(cleaned_data[num_cols], ax=ax[0])
ax[0].set_ylabel('Original distribution')

scaler = pp.MinMaxScaler()
nums_scaled = pd.DataFrame(scaler.fit_transform(cleaned_data[num_cols]),
                           columns=scaler.feature_names_in_)

sns.kdeplot(nums_scaled, ax=ax[1])
ax[1].set_ylabel('Scaled distribution')

fig.show()

In [ ]:
cleaned_data[num_cols] = nums_scaled

cleaned_data.info()

## **Моделирование**

### Деление данных

In [ ]:
X_eval = (cleaned_data
          .query('target == 0')
          .drop('target', axis=1))

cleaned_data.drop(X_eval.index, axis=0, inplace=True)

X = cleaned_data.drop(['target'], axis=1)
y = cleaned_data['target']

X_train, X_test, y_train, y_test = ms.train_test_split(X, y, 
                                                       test_size=0.2, 
                                                       random_state=R)


X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_eval.shape

### Отбор самых сильных признаков

In [ ]:
selector = fs.SelectKBest(score_func=fs.f_regression, k=20)
selector.fit(X_train, y_train)

best_feats = selector.get_feature_names_out()

X_train = X_train[best_feats]
X_test = X_test[best_feats]
X_eval = X_eval[best_feats]

selector_scores = pd.DataFrame({'features': selector.feature_names_in_, 
                                'statistic': selector.scores_})

plt.figure(figsize=(10, 7))
sns.barplot(selector_scores.sort_values('statistic', ascending=False)[:20], 
            y='features', 
            x='statistic', 
            orient='h').set_title('Feature Importance')

plt.show()

### Подбор модели

In [ ]:
models = pd.DataFrame(columns=['MAE_train', 'MAE_test', 'MAE_difference', 
                               'MAPE_train', 'MAPE_test', 'MAPE_difference',
                               'R2_train', 'R2_test', 'R2_difference', 
                               'Model'])


def regression_estimate(model, 
                        X_train=X_train, 
                        y_train=y_train, 
                        X_test=X_test, 
                        y_test=y_test, 
                        params=None):
    metric_dict = {}
    
    if params is not None:
        rs = ms.RandomizedSearchCV(estimator=model, 
                                   param_distributions=params, 
                                   random_state=R, cv=5, n_jobs=-1) 
        rs.fit(X_train, y_train)
        
        model = rs.best_estimator_
    
    else:
        model.fit(X_train, y_train)
    
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    mae = lambda y_true, y_pred: m.mean_absolute_error(y_true, y_pred).round(2)
    mape = lambda y_true, y_pred: m.mean_absolute_percentage_error(y_true, y_pred).round(2)
    
    metric_dict['MAE_train'] = mae(y_train, train_pred)
    metric_dict['MAE_test'] = mae(y_test, test_pred)
    metric_dict['MAE_difference'] = np.abs(metric_dict['MAE_train'] - 
                                           metric_dict['MAE_test'])
    
    metric_dict['MAPE_train'] = mape(y_train, train_pred)
    metric_dict['MAPE_test'] = mape(y_test, test_pred)
    metric_dict['MAPE_difference'] = np.abs(metric_dict['MAPE_train'] - 
                                            metric_dict['MAPE_test'])
    
    metric_dict['R2_train'] = m.r2_score(y_train, train_pred).round(2)
    metric_dict['R2_test'] = m.r2_score(y_test, test_pred).round(2)
    metric_dict['R2_difference'] = np.abs(metric_dict['R2_train'] - 
                                          metric_dict['R2_test'])
    
    metric_dict['Model'] = model
    
    return metric_dict


def add_metrics_model(model_name, metrics, data=models):    
    data.loc[model_name] = metrics
    
    return data


model_loc = lambda model: models.loc[model, 'Model']


models

#### *Линейная регрессия (baseline)*

In [ ]:
model = lm.LinearRegression()
metrics = regression_estimate(model)

add_metrics_model('Linear', metrics)

#### *Полиномиальная регрессия*

In [ ]:
polynom = pp.PolynomialFeatures(degree=2)
polynom.fit(X_train)

X_train_polynom = polynom.transform(X_train)
X_test_polynom = polynom.transform(X_test)

metrics = regression_estimate(model, 
                              X_train=X_train_polynom, 
                              X_test=X_test_polynom)

add_metrics_model('Polynomial', metrics)

### Подбор модели с параметрами

#### *Стохастический градиентный спуск*

In [ ]:
model = lm.SGDRegressor(random_state=R)
metrics = regression_estimate(model)

add_metrics_model('SGD', metrics).loc[['SGD']]

In [ ]:
params = {'max_iter': [1000, 1e4], 
          'learning_rate': ['invscaling', 'constant'], 
          'eta0': [0.01, 0.001, 0.0001]}

metrics = regression_estimate(model, params=params)

add_metrics_model('SGD', metrics)

In [ ]:
model_loc('SGD')

#### *Дерево решений*

In [ ]:
model = tree.DecisionTreeRegressor(random_state=R)
metrics = regression_estimate(model)

add_metrics_model('Decision Tree', metrics).loc[['Decision Tree']]

In [ ]:
params = {'max_depth': [6, 8, 10], 
          'min_samples_split': [1, 2, 3], 
          'min_samples_leaf': [1, 2, 3, 4]}

metrics = regression_estimate(model, params=params)

add_metrics_model('Decision Tree', metrics)

In [ ]:
model_loc('Decision Tree')

#### *Случайный лес*

In [ ]:
model = ens.RandomForestRegressor(random_state=R, n_jobs=-1)
metrics = regression_estimate(model)

add_metrics_model('Random Forest', metrics).loc[['Random Forest']]

In [ ]:
model = ens.RandomForestRegressor(n_estimators=500, 
                                  max_depth=10,
                                  min_samples_leaf=3,
                                  random_state=R, n_jobs=-1)

metrics = regression_estimate(model)

add_metrics_model('Random Forest', metrics)

In [ ]:
model_loc('Random Forest')

#### *Стекинг*

In [ ]:
estimators = [('lr', model_loc('Linear')), 
              ('sgd', model_loc('SGD')), 
              ('dt', model_loc('Decision Tree'))]
final_estimator = ens.RandomForestRegressor(max_depth=10, random_state=R)

model = ens.StackingRegressor(estimators=estimators, 
                              final_estimator=final_estimator, 
                              cv=5, n_jobs=-1)

metrics = regression_estimate(model)

add_metrics_model('Stacking', metrics)

In [ ]:
model_loc('Stacking') 

### Бустинг

#### *Адаптивный бустинг*

In [ ]:
model = ens.AdaBoostRegressor(random_state=R)
metrics = regression_estimate(model)

add_metrics_model('Adaptive Boosting', metrics).loc[['Adaptive Boosting']]

In [ ]:
model = ens.AdaBoostRegressor(estimator=model_loc('Decision Tree'), 
                              n_estimators=100, 
                              random_state=R)

metrics = regression_estimate(model)

add_metrics_model('Adaptive Boosting', metrics)

In [ ]:
model_loc('Adaptive Boosting')

#### *Градиентный бустинг*

In [ ]:
model = ens.GradientBoostingRegressor(random_state=R)
metrics = regression_estimate(model)

add_metrics_model('Gradient Boosting', metrics).loc[['Gradient Boosting']]

In [ ]:
model = ens.GradientBoostingRegressor(n_estimators=300, 
                                      criterion='squared_error',
                                      max_depth=6, 
                                      random_state=R)

metrics = regression_estimate(model)

add_metrics_model('Gradient Boosting', metrics)

In [ ]:
model_loc('Gradient Boosting')

#### *Бустинг-регрессия от `xgboost`*

In [ ]:
model = xgb.XGBRegressor()
metrics = regression_estimate(model)

add_metrics_model('XGBoost', metrics).loc[['XGBoost']]

In [ ]:
model = xgb.XGBRegressor(n_estimators=500, 
                         max_depth=6, 
                         eta=0.1,
                         n_jobs=-1)

metrics = regression_estimate(model)

add_metrics_model('XGBoost', metrics)

In [ ]:
model_loc('XGBoost')

#### *Регрессия от `LightGBM`*

In [ ]:
model = lgbm.LGBMRegressor(random_state=R, n_jobs=-1)
metrics = regression_estimate(model)

add_metrics_model('LightGBM', metrics).loc[['LightGBM']]

In [ ]:
params = {'max_depth': [3, 5, 10], 
          'n_estimators': [100, 300, 500], 
          'learning_rate': [0.1, 0.01]} 

metrics = regression_estimate(model, params=params)

add_metrics_model('LightGBM', metrics)

In [ ]:
model_loc('LightGBM')

#### *Регрессия от `CatBoost`*

In [ ]:
model = cb.CatBoostRegressor(random_state=R, verbose=False)
metrics = regression_estimate(model)

add_metrics_model('CatBoost', metrics).loc[['CatBoost']]

In [ ]:
model = cb.CatBoostRegressor(learning_rate=0.01, 
                             max_depth=10,
                             n_estimators=1000,  
                             eval_metric='MAE', 
                             random_state=R, verbose=False)

metrics = regression_estimate(model)

add_metrics_model('CatBoost', metrics)

In [ ]:
model_loc('CatBoost').get_all_params()

### Прочие модели

#### *Экстра-дерево*

In [ ]:
model = tree.ExtraTreeRegressor(random_state=R)
metrics = regression_estimate(model)

add_metrics_model('Extra Tree', metrics).loc[['Extra Tree']]

In [ ]:
model.set_params(**model_loc('Decision Tree').get_params())
metrics = regression_estimate(model)

add_metrics_model('Extra Tree', metrics)

#### *Экстра-лес*

In [ ]:
model = ens.ExtraTreesRegressor(random_state=R, n_jobs=-1)
metrics = regression_estimate(model)

add_metrics_model('Extra Trees', metrics).loc[['Extra Trees']]

In [ ]:
model.set_params(**model_loc('Random Forest').get_params())
metrics = regression_estimate(model)

add_metrics_model('Extra Trees', metrics)

#### *Голосующая регрессия*

In [ ]:
estimators = [('et', model_loc('Extra Tree')), 
              ('ef', model_loc('Extra Trees'))]

model = ens.VotingRegressor(estimators=estimators, n_jobs=-1) 
metrics = regression_estimate(model)

add_metrics_model('Voting', metrics)

### Выбор модели и улучшение ее качества

In [ ]:
most_accurate_model  = (models
                        .query('(R2_test >= 0.5) & (MAPE_test <= 5)')
                        .sort_values('MAPE_test')
                        .iloc[0]['Model']) 

most_accurate_model

In [ ]:
least_overfitted_model = (models
                          .query('(R2_test >= 0.5) & (MAPE_test <= 5)')
                          .sort_values('MAE_difference')
                          .iloc[0]['Model'])

least_overfitted_model

In [ ]:
params={"learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
        "max_depth": [3, 4, 5, 6, 8, 10, 12, 15],
        "min_child_weight": [1, 3, 5, 7],
        "gamma": [0.0, 0.1, 0.2 , 0.3, 0.4],
        "colsample_bytree": [0.3, 0.4, 0.5 , 0.7]}

metrics = regression_estimate(model, params=params)

metrics